# Build pytorch LSTM

In [1]:
import pandas as pd
from pathlib import Path
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Union, Optional

import matplotlib as mpl
import seaborn as sns
mpl.rcParams['figure.dpi'] = 150

import os

# set the working directory
current_path = Path('.').resolve()
if current_path == Path('/home/tommy/ml_drought/notebooks/draft'):
    os.chdir(current_path.parents[1].as_posix())

current_path = Path('.').resolve()
assert current_path.name == "ml_drought"

%load_ext autoreload
%autoreload 2

In [3]:
from scripts.utils import get_data_path
from src.engineer import Engineer
from src.utils import get_ds_mask
from src.utils import drop_nans_and_flatten

In [4]:
data_dir = get_data_path()
EXPERIMENT = experiment = "one_month_forecast"

In [6]:
preprocessed_dir = data_dir / "interim/mantle_modis_preprocessed"
ds = xr.open_dataset(preprocessed_dir / "data_india.nc")

In [7]:
experiment = "one_month_forecast"
process_static = True

engineer = Engineer(
    data_dir, experiment=experiment, process_static=process_static
)
all_data = engineer.engineer_class._make_dataset(static=False)

Processing /cats/datastore/data/interim/VCI_preprocessed/data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/volumetric_soil_water_layer_1_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/volumetric_soil_water_layer_2_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/volumetric_soil_water_layer_3_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/volumetric_soil_water_layer_4_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/potential_evaporation_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/total_precipitation_data_india.nc
Processing /cats/datastore/data/interim/reanalysis-era5-land-monthly-means_preprocessed/2m_temperature_data_india.nc
Processing /cats/datastore/data/interi

In [8]:
kenya = xr.open_dataset(data_dir / "for_andreas/train_test/ALL_POSSIBLE_INPUT_DATA.nc")
ndvi = xr.open_dataset(data_dir / "for_andreas/ndvi_20012019.nc")
df = kenya.to_dataframe()
n = ndvi.to_dataframe()
data = n.join(df.drop(n.columns, axis=1))

# My datamodule

In [86]:
import sys
sys.path.append("/home/tommy/neuralhydrology")

from neuralhydrology.datasetzoo.basedataset import BaseDataset

In [99]:
dd = (data_dir / "for_andreas/train_test/ALL_POSSIBLE_INPUT_DATA.nc")
kenya

<xarray.Dataset>
Dimensions:     (lat: 45, lon: 35, time: 205)
Coordinates:
  * time        (time) datetime64[ns] 2001-12-31 2002-01-31 ... 2018-12-31
  * lat         (lat) float32 6.0 5.75 5.5 5.25 5.0 ... -4.25 -4.5 -4.75 -5.0
  * lon         (lon) float32 33.75 34.0 34.25 34.5 ... 41.5 41.75 42.0 42.25
Data variables:
    VCI3M       (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    boku_VCI    (time, lat, lon) float64 43.72 31.13 85.88 20.18 ... nan nan nan
    modis_ndvi  (time, lat, lon) float64 0.4688 0.4032 0.624 ... nan nan nan
    VCI         (time, lat, lon) float64 39.64 37.79 89.79 53.54 ... nan nan nan
    precip      (time, lat, lon) float64 13.66 18.42 24.78 47.38 ... nan nan nan
    E           (time, lat, lon) float64 52.86 64.59 64.59 75.49 ... nan nan nan
    Eb          (time, lat, lon) float64 2.3 2.311 2.311 2.409 ... nan nan nan
    SMroot      (time, lat, lon) float64 0.2993 0.3273 0.3273 ... nan nan nan
    SMsurf      (time, lat, lon) float64 0.2837 0.309 0.309 ... nan nan nan
    p84.162     (time, lat, lon) float64 1.121e-05 3.69e-05 ... nan nan
    pev         (time, lat, lon) float64 -0.007123 -0.007165 ... nan nan
    sp          (time, lat, lon) float64 9.591e+04 9.604e+04 ... nan nan
    t2m         (time, lat, lon) float64 303.0 303.8 302.2 299.3 ... nan nan nan
    tp          (time, lat, lon) float64 0.0004198 0.0003483 ... nan nan

# PREPARATION

In [232]:
from tqdm import tqdm


def _join_latlon_cols(df: pd.DataFrame) -> pd.Series:
    points = df.astype({"lat": "str", "lon": "str"})
    return points['lat'] + "_" +  points['lon']

def _latlon_as_strings(ds):
    points = ds[["lat", "lon"]].to_dataframe().reset_index()
    return _join_latlon_cols(points)

def write_latlons_to_file(ds, data_dir: Path) -> Path:
    itemlist = _latlon_as_strings(ds).values
    with open(data_dir / "pixels.csv", "w") as outfile:
        outfile.write("\n".join(str(item) for item in itemlist))

    return data_dir / "pixels.csv"
    

def create_csv_pixels_from_ds(parent_dir: Path, ds: xr.Dataset) -> None:
    if not parent_dir.exists():
        parent_dir.mkdir(exist_ok=True)

    df = ds.to_dataframe().reset_index()
    df["pixel"] = _join_latlon_cols(df)
    for pix in tqdm(df["pixel"].unique()):
        df.loc[df["pixel"] == pix]
        d = df.loc[df["pixel"] == pix]
        d.to_csv(parent_dir / f"{pix}.csv")

# create a file of the pixels to test
fp = write_latlons_to_file(kenya, data_dir)

# create csv files for each pixel ("basin")
parent_dir = data_dir / "KENYA"

if len(list(parent_dir.glob("*.csv"))) != 1575:
    create_csv_pixels_from_ds(parent_dir, kenya)
else:
    print("Already created")

Already created


In [312]:
from neuralhydrology.utils.config import Config
import io 

cfg = io.StringIO("""
# --- Experiment configurations --------------------------------------------------------------------
# experiment name, used as folder name
experiment_name: xarray

# place to store run directory (if empty runs are stored in code_dir/runs/)
run_dir: /cats/datastore/data/runs/

# files to specify training, validation and test basins (relative to code root or absolute path)
train_basin_file: /cats/datastore/data/pixels.csv
validation_basin_file: /cats/datastore/data/pixels.csv
test_basin_file: /cats/datastore/data/pixels.csv

# training, validation and test time periods (format = 'dd/mm/yyyy')
train_start_date: '01/01/2001'
train_end_date: '31/12/2015'
validation_start_date: '01/10/1975'
validation_end_date: '30/09/1980'
test_start_date: '01/01/2016'
test_end_date: '31/12/2020'

# which GPU (id) to use [in format of cuda:0, cuda:1 etc, or cpu or None]
device: cuda:0

# fixed seed, leave empty to use a random seed
seed: 1234

# --- Validation configuration ---------------------------------------------------------------------

# specify after how many epochs to perform validation
validate_every: 5

# specify how many random basins to use for validation
validate_n_random_basins: 1

# specify which metrics to calculate during validation (see neuralhydrology.evaluation.metrics)
# this can either be a list or a dictionary. If a dictionary is used, the inner keys must match the name of the
# target_variable specified below. Using dicts allows for different metrics per target variable.
metrics:
- NSE

# --- Model configuration --------------------------------------------------------------------------

# base model type [lstm, ealstm, cudalstm, embcudalstm, mtslstm]
# (has to match the if statement in modelzoo/__init__.py)
model: ealstm

# prediction head [regression]. Define the head specific parameters below
head: regression

# ----> Regression settings <----
output_activation: linear

# ----> General settings <----

# Number of cell states of the LSTM
hidden_size: 64

# Initial bias value of the forget gate
initial_forget_bias: 3

# Dropout applied to the output of the LSTM
output_dropout: 0.4

# ----> Embedding network settings <----
# define number of neurons per layer in the FC network used as embedding/shortcut network
embedding_hiddens:
- 30
- 20
- 64

# activation function of embedding network
embedding_activation: tanh

# dropout applied to embedding network
embedding_dropout: 0.2

# --- Training configuration -----------------------------------------------------------------------

# specify optimizer [Adam]
optimizer: Adam

# specify loss [MSE, NSE, RMSE]
loss: NSE

# specify learning rates to use starting at specific epochs (0 is the initial learning rate)
learning_rate:
    0: 1e-3
    10: 5e-4
    20: 1e-4

# Mini-batch size
batch_size: 256

# Number of training epochs
epochs: 5

# If a value, clips the gradients during training to that norm.
clip_gradient_norm: 1

# Defines which time steps are used to calculate the loss. Can't be larger than seq_length.
# If use_frequencies is used, this needs to be a dict mapping each frequency to a predict_last_n-value, else an int.
predict_last_n: 1

# Length of the input sequence
# If use_frequencies is used, this needs to be a dict mapping each frequency to a seq_length, else an int.
seq_length: 3

# Number of parallel workers used in the data pipeline
num_workers: 8

# Log the training loss every n steps
log_interval: 10

# If true, writes logging results into tensorboard file
log_tensorboard: True

# If a value and greater than 0, logs n random basins as figures during validation
log_n_figures: 1

# Save model weights every n epochs
save_weights_every: 1


# --- Data configurations --------------------------------------------------------------------------

# which data set to use [camels_us, camels_gb, global, hourly_camels_us]
dataset: xarray

# Path to data set root
data_dir: /cats/datastore/data/KENYA

# whether to load the entire data into memory or not
cache_validation_data: True

# which columns to use as target
target_variables:
- boku_VCI

# variables to use as time series input (names match the data file column headers)
dynamic_inputs:
- VCI3M
- t2m
- precip

# LAGGED INPUTS
lagged_features:

# Which CAMELS attributes to use. Leave empty if none should be used
# static_inputs:
camels_attributes:

# clip negative predictions to zero for all variables listed below. Should be a list, even for single variables.
clip_target_to_zero:
- boku_VCI

# zero_center_target: True -- DEPRECEATED
custom_normalization:

use_basin_id_encoding: True
""")

with open(data_dir / 'file.yml', mode='w') as f:
    print(cfg.getvalue(), file=f)

run_dir = data_dir / "runs/ensemble_EALSTM/ealstm_ensemble6_nse_1998_2008_2910_030601"
config_path = (data_dir / "file.yml") 
config = Config(config_path)

config.train_dir = config.run_dir / "train_data"
config.train_dir.mkdir(parents=True, exist_ok=True)

# DATASET

In [313]:
from typing import List, Dict, Union
from pathlib import Path
import pandas as pd
import xarray as xr

from neuralhydrology.datasetzoo.basedataset import BaseDataset
from neuralhydrology.utils.config import Config


def load_xarray_timeseries(data_dir: Path, basin: str) -> pd.DataFrame:
    assert (data_dir / f"{basin}.csv").exists()
    df = pd.read_csv(data_dir / f"{basin}.csv")
    df = df.drop([c for c in df.columns if "Unnamed" in c], axis=1)
    df["time"] = pd.to_datetime(df["time"])
    df = df.set_index("time")
    
    return df


class XarrayDataset(BaseDataset):
    """Template class for adding a new data set.

    Each dataset class has to derive from `BaseDataset`, which implements most of the logic for preprocessing data and
    preparing data for model training. Only two methods have to be implemented for each specific dataset class:
    `_load_basin_data()`, which loads the time series data for a single basin, and `_load_attributes()`, which loads
    the static attributes for the specific data set.

    Usually, we outsource the functions to load the time series and attribute data into separate functions (in the
    same file), which we then call from the corresponding class methods. This way, we can also use specific basin data
    or dataset attributes without these classes.

    To make this dataset available for model training, don't forget to add it to the `get_dataset()` function in
    'neuralhydrology.datasetzoo.__init__.py'

    Parameters
    ----------
    cfg : Config
        The run configuration.
    is_train : bool
        Defines if the dataset is used for training or evaluating. If True (training), means/stds for each feature
        are computed and stored to the run directory. If one-hot encoding is used, the mapping for the one-hot encoding
        is created and also stored to disk. If False, a `scaler` input is expected and similarly the `id_to_int` input
        if one-hot encoding is used.
    period : {'train', 'validation', 'test'}
        Defines the period for which the data will be loaded
    basin : str, optional
        If passed, the data for only this basin will be loaded. Otherwise the basin(s) are read from the appropriate
        basin file, corresponding to the `period`.
    additional_features : List[Dict[str, pd.DataFrame]], optional
        List of dictionaries, mapping from a basin id to a pandas DataFrame. This DataFrame will be added to the data
        loaded from the dataset, and all columns are available as 'dynamic_inputs', 'static_inputs' and
        'target_variables'
    id_to_int : Dict[str, int], optional
        If the config argument 'use_basin_id_encoding' is True in the config and period is either 'validation' or
        'test', this input is required. It is a dictionary, mapping from basin id to an integer (the one-hot encoding).
    scaler : Dict[str, Union[pd.Series, xarray.DataArray]], optional
        If period is either 'validation' or 'test', this input is required. It contains the means and standard
        deviations for each feature and is stored to the run directory during training (train_data/train_data_scaler.p)

    """

    def __init__(self,
                 cfg: Config,
                 is_train: bool,
                 period: str,
                 basin: str = None,
                 additional_features: List[Dict[str, pd.DataFrame]] = [],
                 id_to_int: Dict[str, int] = {},
                 scaler: Dict[str, Union[pd.Series, xr.DataArray]] = {}):
        # initialize parent class
        super(XarrayDataset, self).__init__(cfg=cfg,
                                            is_train=is_train,
                                            period=period,
                                            basin=basin,
                                            additional_features=additional_features,
                                            id_to_int=id_to_int,
                                            scaler=scaler)

    def _load_basin_data(self, basin: str) -> pd.DataFrame:
        """Load basin time series data

        This function is used to load the time series data (meteorological forcing, streamflow, etc.) and make available
        as time series input for model training later on. Make sure that the returned dataframe is time-indexed.

        Parameters
        ----------
        basin : str
            Basin identifier as string.

        Returns
        -------
        pd.DataFrame
            Time-indexed DataFrame, containing the time series data (e.g., forcings + discharge).
        """
        df = load_xarray_timeseries(data_dir=self.cfg.data_dir, basin=basin)
        return df



    def _load_attributes(self) -> pd.DataFrame:
        """Load dataset attributes

        This function is used to load basin attribute data (e.g. CAMELS catchments attributes) as a basin-indexed
        dataframe with features in columns.

        Returns
        -------
        pd.DataFrame
            Basin-indexed DataFrame, containing the attributes as columns.
        """
        ########################################
        # Add your code for data loading here. #
        ########################################
        pass
    
if "dataset" not in globals().keys():
    dataset = XarrayDataset(config, is_train=True, period="train")
# dataset = XarrayDataset(config, is_train=True, period="train")
    
if "created_ds" not in globals().keys():
    created_ds = dataset._load_or_create_xarray_dataset()
# created_ds = dataset._load_or_create_xarray_dataset()

  1%|          | 8/1575 [00:00<00:21, 72.34it/s]

/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.py:132: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)


  1%|          | 13/1575 [00:00<00:12, 126.25it/s]

/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.py:132: FutureWarning: Units 'M', 'Y' and 'y' do not rep

  2%|▏         | 39/1575 [00:00<00:12, 126.33it/s]

...

 99%|█████████▉| 1560/1575 [00:12<00:00, 124.33it/s]

/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.py:132: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.py:132: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.py:132: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.p

  0%|          | 7/1575 [00:00<00:22, 69.68it/s]

/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.py:132: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.py:132: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.py:132: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)
/cats/datastore/data/minconda3/envs/pl/lib/python3.8/site-packages/pandas/core/tools/timedeltas.p

100%|██████████| 1575/1575 [00:21<00:00, 72.32it/s]


In [353]:
check = created_ds.isnull().mean().to_array().values
assert all(np.ones_like(check) != check), "Every value is null!"

AssertionError: Every value is null!

# Try a new one

In [374]:
sys.path.append("/home/tommy/pytorch-forecasting")


In [375]:
kenya

<xarray.Dataset>
Dimensions:     (lat: 45, lon: 35, time: 205)
Coordinates:
  * time        (time) datetime64[ns] 2001-12-31 2002-01-31 ... 2018-12-31
  * lat         (lat) float32 6.0 5.75 5.5 5.25 5.0 ... -4.25 -4.5 -4.75 -5.0
  * lon         (lon) float32 33.75 34.0 34.25 34.5 ... 41.5 41.75 42.0 42.25
Data variables:
    VCI3M       (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    boku_VCI    (time, lat, lon) float64 43.72 31.13 85.88 20.18 ... nan nan nan
    modis_ndvi  (time, lat, lon) float64 0.4688 0.4032 0.624 ... nan nan nan
    VCI         (time, lat, lon) float64 39.64 37.79 89.79 53.54 ... nan nan nan
    precip      (time, lat, lon) float64 13.66 18.42 24.78 47.38 ... nan nan nan
    E           (time, lat, lon) float64 52.86 64.59 64.59 75.49 ... nan nan nan
    Eb          (time, lat, lon) float64 2.3 2.311 2.311 2.409 ... nan nan nan
    SMroot      (time, lat, lon) float64 0.2993 0.3273 0.3273 ... nan nan nan
    SMsurf      (time, lat, lon) float64 0.2837 0.309 0.309 ... nan nan nan
    p84.162     (time, lat, lon) float64 1.121e-05 3.69e-05 ... nan nan
    pev         (time, lat, lon) float64 -0.007123 -0.007165 ... nan nan
    sp          (time, lat, lon) float64 9.591e+04 9.604e+04 ... nan nan
    t2m         (time, lat, lon) float64 303.0 303.8 302.2 299.3 ... nan nan nan
    tp          (time, lat, lon) float64 0.0004198 0.0003483 ... nan nan

In [376]:
import pickle
pickle.dump(kenya.copy(), (data_dir / "kenya.pkl").open("wb"))

In [367]:
print(dir(config))

p = dict(
    target_variable="boku_VCI",
    seed=1,
    dynamic_inputs=["VCI3M", "t2m", "precip"],
    lagged_features=None,
    seq_length=3,
)

In [368]:
class LSTMRegressor(pl.LightningModule):
    '''
    Standard PyTorch Lightning module:
    https://pytorch-lightning.readthedocs.io/en/latest/lightning_module.html
    '''
    def __init__(self, 
                 n_features, 
                 hidden_size, 
                 seq_len, 
                 batch_size,
                 num_layers, 
                 dropout, 
                 learning_rate,
                 criterion):
        super(LSTMRegressor, self).__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.criterion = criterion
        self.learning_rate = learning_rate

        self.lstm = nn.LSTM(input_size=n_features, 
                            hidden_size=hidden_size,
                            num_layers=num_layers, 
                            dropout=dropout, 
                            batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        # lstm_out = (batch_size, seq_len, hidden_size)
        h_s, c_s = self.lstm(x)
        y_pred = self.linear(h_s[:,-1])
        return y_pred
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def training_step(s elf, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('test_loss', loss)
        return loss
